In [ ]:
# Importación de bibliotecas necesarias para la manipulación de datos y configuración del proyecto.
# pandas se utiliza para la manipulación de dataframes, numpy para operaciones numéricas,
# y os para la interacción con el sistema de archivos.
import pandas as pd
import numpy as np
import os
from configuracion import configuracion as cf  # Importación de configuraciones personalizadas del proyecto.


# Generación población sintética base.

### Genero, edad, cantón, región

In [ ]:
# Carga de datos del archivo CSV 'CantonEdad.csv', que contiene la distribución de población
# por edad y género para cada cantón. Se utiliza la configuración del directorio desde un módulo
# de configuración para definir la ruta del archivo. Se muestran las primeras 20 filas para una revisión inicial.
df_cantonEdad = pd.read_csv(cf.directorio_data + 'CantonEdad.csv', sep=',', decimal='.')
df_cantonEdad.head(20)


In [ ]:
# Definición de una función para simular las características de una persona basada en 
# la distribución por edad y género en un cantón específico.
def simular_persona(datos_cantón):
    # Calcular la probabilidad de que la persona sea hombre, basada en la proporción 
    # de hombres respecto al total de la población del cantón.
    prob_hombre = datos_cantón['Hombres'].values[0] / datos_cantón['Total General'].values[0]
    # Asignar género basado en la probabilidad calculada.
    género = 'Hombre' if np.random.rand() < prob_hombre else 'Mujer'

    # Seleccionar grupo de edad según las probabilidades proporcionales a la población 
    # de cada grupo de edad para hombres o mujeres.
    grupos_edad = ['De 0 a 14 años', 'De 15 a 24 años', 'De 25 a 34 años', 'De 35 a 44 años', 
                   'De 45 a 59 años', 'De 60 años o más']
    if género == 'Hombre':
        poblaciones = [datos_cantón[f'Hombres {edad}'].values[0] for edad in grupos_edad]
    else:
        poblaciones = [datos_cantón[f'Mujeres {edad}'].values[0] for edad in grupos_edad]

    # Seleccionar aleatoriamente un grupo de edad basado en la distribución proporcional.
    grupo_edad = np.random.choice(grupos_edad, p=np.array(poblaciones) / np.sum(poblaciones))

    # Devolver el género y el grupo de edad de la persona simulada.
    return género, grupo_edad

# Creación de un DataFrame para almacenar la población sintética por cantón.
sint_poblacion_por_canton = pd.DataFrame()
for cantón_interes in list(df_cantonEdad['Canton']):
    # Filtrar los datos para el cantón de interés.
    df_canton = df_cantonEdad[df_cantonEdad['Canton'] == cantón_interes]
    # Determinar el número de personas a simular en función del total general del cantón.
    total_simulaciones_por_canton = df_canton.iloc[0,4] # Se puede ajustar según necesidades del análisis.

    # Simular las características de cada persona en el cantón usando la función definida.
    personas_simuladas = [simular_persona(df_canton) for _ in range(total_simulaciones_por_canton)]

    # Convertir los resultados de la simulación en un DataFrame para facilitar el análisis.
    canton_simulado = pd.DataFrame(personas_simuladas, columns=['Género', 'Rango edad'])
    canton_simulado['Canton'] = cantón_interes

    # Concatenar los resultados de cada cantón en un DataFrame general.
    sint_poblacion_por_canton = pd.concat([sint_poblacion_por_canton, canton_simulado])

# Adición de la dimensión de región a la población sintética, vinculando los datos del cantón con la región correspondiente.
sint_poblacion_por_region = pd.merge(df_cantonEdad[['Región', 'Canton']], 
                                     sint_poblacion_por_canton, 
                                     how='inner', on='Canton')


# Asignación de status de ocupación

### Segun genero y edad

Dado que no tienes un identificador único y ambos datasets son simulaciones basadas en distribuciones de variables compartidas (edad y género), puedes combinar estos conjuntos de datos utilizando un enfoque probabilístico que respete las distribuciones subyacentes en cada dataset. Aquí te explico una estrategia posible:

Estrategia de Combinación Probabilística
Normalización de Datasets:

Cálculo de Probabilidades Condiciones:
Para el dataset con estado laboral, se calcula la probabilidad condicional de cada estado laboral dado el género y la edad, igual a una tabla de probabilidad P(Estado Laboral | Edad, Género).

Asignación de Estado Laboral:
Para cada individuo en el primer dataset (el que contiene edad, género y región), utiliza las probabilidades calculadas para simular un estado laboral, basado en la edad y el género de cada individuo, se consulta la tabla de probabilidad P(Estado Laboral | Edad, Género) y genera el estado laboral de acuerdo con las probabilidades asociadas.
Este proceso se realiza mediante un muestreo aleatorio donde cada estado laboral se selecciona de acuerdo a su probabilidad condicional.

Simulación y Asignación:
Se utiliza un generador de números aleatorios para realizar el muestreo basado en las probabilidades. Cada selección es independiente y respetará las distribuciones observadas en el segundo dataset.
La asignación se hace de manera que mantenga la coherencia estadística general, respetando las distribuciones marginales y conjuntas observadas.

Este método de combinación probabilística permite integrar dos conjuntos de datos sintéticos que no tienen un identificador común pero comparten algunas variables clave, asegurando que la integración respete las distribuciones originales y proporcionando un resultado final coherente y estadísticamente valido.

In [ ]:
# Generacion del segundo dataset
df_edad_genero_status = pd.read_csv(cf.directorio_data + 'EdadGeneroStatus.csv')
df_edad_genero_status.round(0)

In [ ]:
# Definición de una función para calcular las probabilidades condicionales de cada estado laboral
# (Ocupada, Desempleada, Fuera de la Fuerza Laboral) por género y grupo de edad.
def calcular_probabilidades_status(df):
    # Crear una lista para acumular los resultados de las probabilidades calculadas.
    resultados = []

    # Iterar sobre cada fila del DataFrame para calcular probabilidades por combinación de edad y género.
    for _, row in df.iterrows():
        edad = row['Rango edad']
        for genero in ['Hombres', 'Mujeres']:
            total = row[f'{genero}_Total']  # Total de individuos por género.
            # Calcular la probabilidad de cada estado laboral para el género y grupo de edad actual.
            for estado in ['Ocupada', 'Desempleada', 'Fuera Fuerza Laboral']:
                count = row[f'{genero}_{estado}']  # Conteo de individuos en cada estado laboral.
                # Cálculo de la probabilidad condicional, evitando divisiones por cero.
                probabilidad = count / total if total > 0 else 0
                # Almacenar el resultado en un diccionario para añadirlo a la lista de resultados.
                resultados.append({
                    'Rango edad': edad,
                    'Género': 'Hombre' if genero == 'Hombres' else 'Mujer',
                    'Estado Laboral': estado,
                    'Probabilidad': probabilidad
                })

    # Convertir la lista de resultados en un DataFrame para análisis posterior.
    return pd.DataFrame(resultados)

# Aplicación de la función para calcular las probabilidades condicionales de estado laboral por grupo de edad y género.
probabilidades = calcular_probabilidades_status(df_edad_genero_status)

# Crear una tabla pivote para organizar las probabilidades por grupo de edad y género,
# con columnas representando cada estado laboral y sus respectivas probabilidades.
tabla_probabilidades_status = probabilidades.pivot_table(
    index=['Rango edad', 'Género'], 
    columns='Estado Laboral', 
    values='Probabilidad', 
    fill_value=0
)
# Definición de una función para simular el estado laboral de una persona basada en las probabilidades calculadas.
def simular_estado_laboral(row):
    edad, genero = row['Rango edad'], row['Género']
    
    # Verificar si la combinación de edad y género está presente en la tabla de probabilidades.
    if (edad, genero) in tabla_probabilidades_status.index:
        # Obtener la distribución de probabilidades para los estados laborales correspondientes a la edad y género.
        distribucion = tabla_probabilidades_status.loc[(edad, genero)]
        estados = distribucion.index  # Lista de estados laborales.
        probabilidades_estado = distribucion.values  # Probabilidades asociadas a cada estado laboral.
        probabilidades_estado /= probabilidades_estado.sum()  # Normalizar las probabilidades para asegurar que sumen a 1.
        
        # Seleccionar aleatoriamente un estado laboral basado en las probabilidades normalizadas.
        return np.random.choice(estados, p=probabilidades_estado)
    else:
        # Retornar 'Desconocido' si no se encuentra la combinación de edad y género en el dataset de probabilidades.
        return 'Desconocido'

# Aplicar la función de simulación de estado laboral al DataFrame de población sintética por región.
sint_poblacion_por_region['Estado Laboral'] = sint_poblacion_por_region.apply(simular_estado_laboral, axis=1)


In [ ]:
sint_poblacion_por_region.groupby(['Rango edad', 'Estado Laboral']).size()

In [ ]:
sint_poblacion_por_region['Rango edad'].value_counts(normalize=True)

# Asignación zona

In [ ]:
# Leer el archivo CSV que contiene la población por zona
df_zonas = pd.read_csv(cf.directorio_data + 'PoblacionPorZona.csv').iloc[:-1,:]
df_zonas

In [ ]:
# Calcular el total de la población por cada rango de edad sumando los valores de las zonas urbana y rural.
df_zonas['Total'] = df_zonas['Urbano'] + df_zonas['Rural']

# Calcular las probabilidades para cada zona (Urbano y Rural) por cada rango de edad.
# Estas probabilidades reflejan la proporción de la población que vive en zonas urbanas y rurales.
df_zonas['Prob_Urbano'] = df_zonas['Urbano'] / df_zonas['Total']
df_zonas['Prob_Rural'] = df_zonas['Rural'] / df_zonas['Total']


# Crear una tabla que contenga las probabilidades de que una persona de un rango de edad dado
# resida en una zona urbana o rural, y establecer el rango de edad como índice para facilitar el acceso.
tabla_probabilidades_zona = df_zonas[['Rango edad', 'Prob_Urbano', 'Prob_Rural']]
tabla_probabilidades_zona.set_index('Rango edad', inplace=True)


# Definición de una función para simular la zona de residencia (Urbano o Rural) de una persona
# basada en su rango de edad utilizando las probabilidades calculadas anteriormente.
def simular_zonas(row):
    edad = row['Rango edad']
    
    # Verificar si el rango de edad está presente en la tabla de probabilidades.
    if edad in tabla_probabilidades_zona.index:
        # Obtener las probabilidades correspondientes a la zona urbana y rural para la edad dada.
        distribucion = tabla_probabilidades_zona.loc[edad]
        zonas = ['Urbano', 'Rural']  # Definir las posibles zonas.
        probabilidades_zona = [distribucion['Prob_Urbano'], distribucion['Prob_Rural']]  # Probabilidades asociadas.
        
        # Seleccionar aleatoriamente una zona basado en las probabilidades calculadas.
        return np.random.choice(zonas, p=probabilidades_zona)
    else:
        # Retornar 'Desconocido' si no se encuentra el rango de edad en la tabla de probabilidades.
        return 'Desconocido'

# Aplicar la función de simulación de zona de residencia al DataFrame de población sintética por región.
sint_poblacion_por_region['Zona'] = sint_poblacion_por_region.apply(simular_zonas, axis=1)

# Comparar la distribución de zonas generada por el modelo con la distribución real de los datos.
# Se calcula la distribución generada a partir del dataset sintético y se compara con la distribución
# real calculada como el promedio de las probabilidades de las zonas urbana y rural en el dataset original.
distribucion_generada = sint_poblacion_por_region['Zona'].value_counts(normalize=True)  # Distribución generada.
distribucion_real = df_zonas[['Prob_Urbano', 'Prob_Rural']].mean()  # Distribución real promedio.

# Imprimir la distribución generada y la distribución real para evaluar la similitud entre ambas.
print("Distribución generada:")
print(distribucion_generada)

print("\nDistribución real:")
print(distribucion_real)


In [ ]:
# Seperación de población sintetica por status laboral
poblacion_ocupada = sint_poblacion_por_region[
                    sint_poblacion_por_region['Estado Laboral'] == 'Ocupada']
poblacion_sin_ocupacion = sint_poblacion_por_region[
                    sint_poblacion_por_region['Estado Laboral'] == 'Fuera Fuerza Laboral']
poblacion_no_ocupada = sint_poblacion_por_region[
                    sint_poblacion_por_region['Estado Laboral'] == 'Desempleada']

# Asignación rama económico

In [ ]:
sector = pd.read_csv(cf.directorio_data + 'PoblacionOcupadaPorSector.csv', sep=',', decimal='.')
sector_des = pd.read_csv(cf.directorio_data + 'PoblacionDesOcupadaPorSector.csv', sep=',', decimal='.')

In [ ]:
# Crear una función para asignar la sector económica basada en las probabilidades condicionales solo por género
def asignar_sector_por_genero(genero):
    probabilidades = sector[genero] / sector[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(sector['Sectoreconómico'], p=probabilidades)


# Asignar la sector económica a cada individuo en el dataset
poblacion_ocupada['Sector'] = poblacion_ocupada['Género'].apply(asignar_sector_por_genero)
print(poblacion_ocupada.head())

In [ ]:
# Crear una función para asignar la sector económica basada en las probabilidades condicionales solo por género
def asignar_sector_des_por_genero(genero):
    probabilidades = sector_des[genero] / sector_des[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(sector_des['Sectoreconómico'], p=probabilidades)

# Asignar la sector económica a cada individuo en el dataset
poblacion_no_ocupada['Sector'] = poblacion_no_ocupada['Género'].apply(asignar_sector_des_por_genero)
print(poblacion_no_ocupada.head())

# Asignacion de sector económico

In [ ]:
rama_sector = pd.read_csv(cf.directorio_data + 'RamaSector.csv', sep=',', decimal='.')
rama_sector

In [ ]:
# Ocupados
poblacion_ocupada = pd.merge(poblacion_ocupada, rama_sector, how='inner', left_on='Sector', right_on='Rama')
poblacion_ocupada = poblacion_ocupada.drop(['Sector_x'], axis=1)
poblacion_ocupada.rename(columns={'Sector_y': 'Sector'}, inplace=True)
print(poblacion_ocupada.head())

In [ ]:
# Desempleados
poblacion_no_ocupada = pd.merge(poblacion_no_ocupada, rama_sector, how='inner', left_on='Sector', right_on='Rama')
poblacion_no_ocupada = poblacion_no_ocupada.drop(['Sector_x'], axis=1)
poblacion_no_ocupada.rename(columns={'Sector_y': 'Sector'}, inplace=True)
print(poblacion_no_ocupada.head())

# Asignación cualificación

In [ ]:
cualificacion = pd.read_csv(cf.directorio_data + 'PoblacionOcupadaPorCalificacion.csv', sep=',', decimal='.')
cualificacion_des = pd.read_csv(cf.directorio_data + 'PoblacionDesOcupadaPorCalificacion.csv', sep=',', decimal='.')

In [ ]:
# Crear una función para asignar la cualificación basada en las probabilidades condicionales solo por género
def asignar_cualificacion_por_genero(genero):
    probabilidades = cualificacion[genero] / cualificacion[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(cualificacion['Nivel calificacion'], p=probabilidades)

# Asignar la rama económica a cada individuo en el dataset
poblacion_ocupada['Nivel calificacion'] = poblacion_ocupada['Género'].apply(asignar_cualificacion_por_genero)
poblacion_ocupada.head()

In [ ]:
# Crear una función para asignar la cualificación basada en las probabilidades condicionales solo por género
def asignar_cualificacion_des_por_genero(genero):
    probabilidades = cualificacion_des[genero] / cualificacion_des[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(cualificacion_des['Nivel calificacion'], p=probabilidades)

# Asignar la rama económica a cada individuo en el dataset
poblacion_no_ocupada['Nivel calificacion'] = poblacion_no_ocupada['Género'].apply(asignar_cualificacion_des_por_genero)
poblacion_no_ocupada.head()

# Asignación institucional

In [ ]:
institucional = pd.read_csv(cf.directorio_data + 'PoblacionOcupadaInstitucional.csv', sep=',', decimal='.')
institucional_des = pd.read_csv(cf.directorio_data + 'PoblacionDesOcupadaInstitucional.csv', sep=',', decimal='.')

In [ ]:
def asignar_institucional_por_genero(genero):
    # Calcular probabilidades acumuladas para el género dado
    probabilidades = institucional[genero] / institucional[genero].sum()
    probabilidades_acumuladas = np.cumsum(probabilidades)
    
    # Generar un número aleatorio
    random_number = np.random.rand()
    
    # Asignar el sector institucional basado en el número aleatorio
    for i, prob_acumulada in enumerate(probabilidades_acumuladas):
        if random_number < prob_acumulada:
            return institucional['Sector Institucional'].iloc[i]

# Asignar la rama económica a cada individuo en el dataset
poblacion_ocupada['Sector Institucional'] = poblacion_ocupada['Género'].apply(asignar_institucional_por_genero)
poblacion_ocupada.head()

In [ ]:
def asignar_institucional_des_por_genero(genero):
    # Calcular probabilidades acumuladas para el género dado
    probabilidades = institucional_des[genero] / institucional_des[genero].sum()
    probabilidades_acumuladas = np.cumsum(probabilidades)
    
    # Generar un número aleatorio
    random_number = np.random.rand()
    
    # Asignar el sector institucional_des basado en el número aleatorio
    for i, prob_acumulada in enumerate(probabilidades_acumuladas):
        if random_number < prob_acumulada:
            return institucional_des['Sector Institucional'].iloc[i]

# Asignar la rama económica a cada individuo en el dataset
poblacion_no_ocupada['Sector Institucional'] = poblacion_no_ocupada['Género'].apply(asignar_institucional_des_por_genero)
poblacion_no_ocupada.head()

# Asignacion de posición

In [ ]:
posicion = pd.read_csv(cf.directorio_data + 'PoblacionOcupadaPorPosicion.csv', sep=',', decimal='.')
posicion_des = pd.read_csv(cf.directorio_data + 'PoblacionDesOcupadaPorPosicion.csv', sep=',', decimal='.')

In [ ]:
# Crear una función para asignar la rama económica basada en las probabilidades condicionales solo por género
def asignar_posicion_por_genero(genero):
    probabilidades = posicion[genero] / posicion[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(posicion['Posicion'], p=probabilidades)

# Asignar la rama económica a cada individuo en el dataset
poblacion_ocupada['Posición'] = poblacion_ocupada['Género'].apply(asignar_posicion_por_genero)
poblacion_ocupada.head()

In [ ]:
# Crear una función para asignar la rama económica basada en las probabilidades condicionales solo por género
def asignar_posicion_des_por_genero(genero):
    probabilidades = posicion_des[genero] / posicion_des[genero].sum()  # Normalizar para que sumen a 1
    return np.random.choice(posicion_des['Posicion'], p=probabilidades)

# Asignar la rama económica a cada individuo en el dataset
poblacion_no_ocupada['Posición'] = poblacion_no_ocupada['Género'].apply(asignar_posicion_des_por_genero)
poblacion_no_ocupada.head()

In [ ]:
poblacion_sintetica = pd.concat([poblacion_ocupada, poblacion_no_ocupada])

In [ ]:
poblacion_sintetica = pd.concat([poblacion_sintetica, poblacion_sin_ocupacion])

In [ ]:
poblacion_sintetica.to_csv(cf.directorio_data + 'PoblacionSintetica.csv', sep=',', decimal='.')